# 🚀 Multi-Agent Systems & Workflow Patterns


In this notebook, you'll:

- ✅ Learn when to use multi-agent systems in [Agent Development Kit (ADK)](https://google.github.io/adk-docs/)
- ✅ Build your first system using an LLM as a "manager"
- ✅ Learn three core workflow patterns (Sequential, Parallel, and Loop) to coordinate your agent teams

---
# 1. Setup

In [ ]:
# get api key
import os
from dotenv import load_dotenv

try:
    load_dotenv()
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

# get dependencies
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

# retry config thats common for all LLM call
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

print("✅ Retry config done.")

✅ Gemini API key setup complete.
✅ ADK components imported successfully.
✅ Retry config done.


---
# 2. Basic Multi-Agent System

Research & Summarization System: 
- First Research 
- Then Summarize
- Sequence is instructed in the Master Agent's prompt

In [2]:
# Research Agent
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings",  # The result of this agent will be stored in the session state with this key.
)

print("✅ research_agent created.")

# Summarizer Agent
summarize_agent = Agent(
    name="SummarizerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary. Be logical and clear. Limit to 200 words.""",
    output_key="current_summary",
)

print("✅ summarize_agent created.")



# Basic Root Agent - Combine both
root_agent_basic = Agent(
    name="Research",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the edited summary clearly to the user as your response.""",
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[AgentTool(research_agent), AgentTool(summarize_agent)],
)

print("✅ basic root_agent created.")

✅ research_agent created.
✅ summarize_agent created.
✅ basic root_agent created.


Here we're using `AgentTool` to wrap the sub-agents to make them callable tools for the root agent. We'll explore `AgentTool` in-detail on Day 2.

Let's run the agent and ask it about a topic:

In [3]:
runner = InMemoryRunner(agent=root_agent_basic)
response = await runner.run_debug(
    "What are the latest advancements in quantum computing and what do they mean for AI? Be concise and no fluff."
)


 ### Created new session: debug_session_id

User > What are the latest advancements in quantum computing and what do they mean for AI? Be concise and no fluff.


/Users/xing.zhang/anaconda3/envs/google-adk/lib/python3.11/site-packages/google/adk/flows/llm_flows/base_llm_flow.py:449: UserWarning: [EXPERIMENTAL] feature FeatureName.PROGRESSIVE_SSE_STREAMING is enabled.
  async for event in agen:


Research > Quantum AI, the synergy between quantum computing and artificial intelligence, promises to revolutionize AI by providing exponentially greater computational power. This will enable AI to tackle previously insurmountable problems in fields like drug discovery and materials science. Quantum computers' ability to process information via qubits will dramatically accelerate AI model training and data processing.

Hybrid quantum-classical systems are expected to maximize the strengths of both technologies. AI is also contributing to quantum computing by improving stability and error mitigation in quantum systems. Significant advancements in quantum hardware, coupled with increasing commercialization and investment, suggest practical applications could emerge by the end of the decade. While not all AI aspects may benefit, the overall outlook for Quantum AI is transformative, with potential to redefine various industries.


### Xing's Notes: Understanding Event Roles in Multi-Agent Systems

| Role | Source | Purpose |
|------|--------|---------|
| `role='model'` | **Root Agent** | The orchestrator's decisions and outputs (function_calls + final response) |
| `role='user'` | **Sub-agents** (ResearchAgent, SummarizerAgent) | Results returned from the tools/sub-agents back to the root agent |

In [4]:
# print out events 
for i, event in enumerate(response):
    print(i)
    print(event.content.role)
    if event.content.parts[0].function_response:
        print(event.content.parts[0].function_response)
    elif event.content.parts[0].function_call:
        print(event.content.parts[0].function_call)
    else:
        print(event.content.parts[0])

0
model
id='adk-32442c8b-f56e-4de9-822d-2a7f67bf4ead' args={'request': 'Latest advancements in quantum computing and their implications for AI'} name='ResearchAgent' partial_args=None will_continue=None
1
user
will_continue=None scheduling=None parts=None id='adk-32442c8b-f56e-4de9-822d-2a7f67bf4ead' name='ResearchAgent' response={'result': "Recent advancements in quantum computing are showing significant promise for revolutionizing artificial intelligence (AI). The synergy between these two fields, often referred to as Quantum AI, is expected to enable the solution of problems previously considered insurmountable for classical computers.\n\nKey developments and their implications include:\n\n*   **Enhanced Computational Power for AI:** Quantum computers, with their use of qubits that can exist in multiple states simultaneously, offer exponentially greater processing power than classical computers. This can dramatically speed up the training of AI models, optimize complex algorithms, a

---

# 3: Sequential Workflows 
**Problem**: LLM-orchestrated multi-agent systems rely on prompts to control execution order, which can be unreliable (steps may be skipped or reordered).

**Solution**: Use SequentialAgent for guaranteed, deterministic order—runs sub-agents as a fixed pipeline where each step's output feeds into the next.

❗ **Note:** In `SequentialAgent`, each sub-agent calls its own LLM directly—there is no root agent orchestrating them. Every sub-agent has `role='model'`. see the print out of events in response for details

In [5]:
root_agent_sequential = SequentialAgent(
    name="ResearchSummarizePipeline",
    sub_agents=[research_agent, summarize_agent],
)

print("✅ Sequential Agent created.")

✅ Sequential Agent created.


In [6]:
runner = InMemoryRunner(agent=root_agent_sequential)
response = await runner.run_debug(
    "What are the latest advancements in quantum computing and what do they mean for AI? Be concise and no fluff."
)


 ### Created new session: debug_session_id

User > What are the latest advancements in quantum computing and what do they mean for AI? Be concise and no fluff.
ResearchAgent > Recent advancements in quantum computing are paving the way for significant leaps in Artificial Intelligence (AI). The core of this progress lies in the development of more stable qubits, improved coherence times, and larger qubit counts, moving from theoretical research to practical applications. These improvements are enabling quantum computers to tackle complex problems that are currently beyond the reach of classical computers, which could revolutionize AI's capabilities.

The fusion of quantum computing and AI, often termed "Quantum AI," promises to make AI models more efficient, faster, and more powerful. Quantum computers utilize qubits, which can exist in multiple states simultaneously due to superposition, allowing them to process a vast number of potential solutions at once. This capability is crucial 

In [7]:
# print out events
for i, event in enumerate(response):
    print(i)
    print(event.content.role)
    if event.content.parts[0].function_response:
        print(event.content.parts[0].function_response)
    elif event.content.parts[0].function_call:
        print(event.content.parts[0].function_call)
    else:
        print(event.content.parts[0])

0
model
media_resolution=None code_execution_result=None executable_code=None file_data=None function_call=None function_response=None inline_data=None text='Recent advancements in quantum computing are paving the way for significant leaps in Artificial Intelligence (AI). The core of this progress lies in the development of more stable qubits, improved coherence times, and larger qubit counts, moving from theoretical research to practical applications. These improvements are enabling quantum computers to tackle complex problems that are currently beyond the reach of classical computers, which could revolutionize AI\'s capabilities.\n\nThe fusion of quantum computing and AI, often termed "Quantum AI," promises to make AI models more efficient, faster, and more powerful. Quantum computers utilize qubits, which can exist in multiple states simultaneously due to superposition, allowing them to process a vast number of potential solutions at once. This capability is crucial for AI, as train

---
# 4. Parallel Workflows

**Problem**: Sequential execution creates bottlenecks when tasks are independent—each step waits unnecessarily for the previous one.

**Solution**: Use ParallelAgent to run independent sub-agents concurrently, then aggregate results in a final step.


In [8]:
english_research_agent = Agent(
    name="EnglishResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information in ENGLISH on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="english_research_findings",  
)

chinese_research_agent = Agent(
    name="ChineseResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information in CHINESE on the given topic and present the findings with citations. return the findings in CHINESE.""",
    tools=[google_search],
    output_key="chinese_research_findings",  
)

parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[english_research_agent, chinese_research_agent],
)


bilingual_summarize_agent = Agent(
    name="BilingualSummarizeAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction="""Combine these two research findings into a single summary:

    **English Research Findings:**
    {english_research_findings}
    
    **Chinese Research Findings:**
    {chinese_research_findings}
    
    Your summary should separately listing the English ones IN ENGLISH and Chinese ones IN CHINESE. Be concise, both should be around 200 words.""",
    output_key="bilingual_summary",  # This will be the final output of the entire system.
)

root_agent_parallel = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_research_team, bilingual_summarize_agent]
)

print("✅ Parallel and Sequential Agents created.")

✅ Parallel and Sequential Agents created.


In [9]:
runner = InMemoryRunner(agent=root_agent_parallel)
response = await runner.run_debug(
    "What are the latest advancements in quantum computing and what do they mean for AI? Be concise and no fluff."
)


 ### Created new session: debug_session_id

User > What are the latest advancements in quantum computing and what do they mean for AI? Be concise and no fluff.
ChineseResearchAgent > Quantum computing advancements are poised to revolutionize AI by overcoming current computational bottlenecks and enabling more sophisticated AI agents. Key developments include:

*   **"Quantum Supremacy"**: This milestone, where quantum computers outperform the most powerful classical supercomputers for specific tasks, signifies the potential for AI agents to tackle previously unimaginable challenges.
*   **Enhanced Data Processing**: Quantum computing can process vast and complex datasets far more efficiently than classical computers, transforming "big data" into "quantum data" for more powerful AI training. Quantum-enhanced machine learning algorithms are expected to be more robust, faster, and more accurate.
*   **Algorithmic Advancements**: Algorithms like Shor's algorithm threaten current encryptio

In [10]:
# print out events
for i, event in enumerate(response):
    print(i)
    print(event.content.role)
    if event.content.parts[0].function_response:
        print(event.content.parts[0].function_response)
    elif event.content.parts[0].function_call:
        print(event.content.parts[0].function_call)
    else:
        print(event.content.parts[0])

0
model
media_resolution=None code_execution_result=None executable_code=None file_data=None function_call=None function_response=None inline_data=None text='Quantum computing advancements are poised to revolutionize AI by overcoming current computational bottlenecks and enabling more sophisticated AI agents. Key developments include:\n\n*   **"Quantum Supremacy"**: This milestone, where quantum computers outperform the most powerful classical supercomputers for specific tasks, signifies the potential for AI agents to tackle previously unimaginable challenges.\n*   **Enhanced Data Processing**: Quantum computing can process vast and complex datasets far more efficiently than classical computers, transforming "big data" into "quantum data" for more powerful AI training. Quantum-enhanced machine learning algorithms are expected to be more robust, faster, and more accurate.\n*   **Algorithmic Advancements**: Algorithms like Shor\'s algorithm threaten current encryption methods, necessitat

---
# 5. Loop Workflows

**Problem**: One-shot workflows (Sequential/Parallel) can't refine poor outputs—they just run once and stop.

**Solution**: Use LoopAgent to repeatedly run sub-agents until a condition is met, enabling iterative feedback and quality improvement.

❗ **Note:** a sub-agent can only have one parent `SequentialAgent`. However, once defined as an agent tool, it can be used by multiple root agents and the agent itself can also add to a sequential parent.

That's why research_agent and summarize_agent will be redefined here.

In [11]:
# Research Agent
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings",  # The result of this agent will be stored in the session state with this key.
)

print("✅ research_agent created.")

# Summarizer Agent
summarize_agent = Agent(
    name="SummarizerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary. Be logical and clear. Limit to 200 words.""",
    output_key="current_summary",
)

print("✅ summarize_agent created.")

# Critique Agent
critic_agent = Agent(
    name="CriticAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a constructive text critic. Review the summary of research findings provided below
    Summary: {current_summary}
    
    Evaluate summary's clarity, logic coherence, and conciseness
    - If the summary is well-written, you MUST respond with the exact phrase: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
    output_key="critique",  # Stores the feedback in the state.
)

print("✅ critic_agent created.")

# This is the function that the RefinerAgent will call to exit the loop.
def exit_loop():
    """Call this function ONLY when the critique is 'APPROVED', indicating the summary is good enough."""
    return {"status": "approved", "message": "Summary approved. Exiting refinement loop."}


print("✅ exit_loop function created.")

✅ research_agent created.
✅ summarize_agent created.
✅ critic_agent created.
✅ exit_loop function created.


In [12]:
# This agent refines the summary based on critique OR calls the exit_loop function.
refiner_agent = Agent(
    name="RefinerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a summary refiner. You have a summary and the critique
    
    Summary: {current_summary}
    Critique: {critique}
    
    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the summary to fully incorporate the feedback from the critique.""",
    output_key="current_summary",  # It overwrites the story with the new, refined version.
    tools=[
        FunctionTool(exit_loop)
    ],  
)

print("✅ refiner_agent created.")

✅ refiner_agent created.


In [13]:
# The LoopAgent contains the agents that will run repeatedly: Critic -> Refiner.
summary_refinement_loop = LoopAgent(
    name="SummaryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2,  # Prevents infinite loops
)

# The root agent is a SequentialAgent that defines the overall workflow: Research -> Summarize -> Refinement Loop
root_agent_loop = SequentialAgent(
    name="RefinePipeline",
    sub_agents=[research_agent, summarize_agent, summary_refinement_loop],
)

print("✅ Loop and Sequential Agents created.")

✅ Loop and Sequential Agents created.


In [14]:
runner = InMemoryRunner(agent=root_agent_loop)
response = await runner.run_debug(
    "What are the latest advancements in quantum computing and what do they mean for AI? Be concise and no fluff."
)


 ### Created new session: debug_session_id

User > What are the latest advancements in quantum computing and what do they mean for AI? Be concise and no fluff.
ResearchAgent > Recent advancements in quantum computing promise to revolutionize AI by offering exponential improvements in processing power and efficiency. These advancements include breakthroughs in quantum hardware, such as the development of the world's first 10,000-qubit quantum processor, and progress in quantum error correction, with the demonstration of a logical qubit prototype.

The implications for AI are significant:

*   **Enhanced AI Capabilities**: Quantum computing can accelerate AI tasks like data mining, pattern recognition, and complex simulations, which are currently limited by classical computing power. This could lead to faster and more accurate AI models, capable of handling exponentially more possibilities for optimization problems.
*   **Solving Complex Problems**: Quantum AI is expected to tackle prob

In [15]:
# print out events
for i, event in enumerate(response):
    print(i)
    print(event.content.role)
    if event.content.parts is not None and event.content.parts[0].function_response:
        print(event.content.parts[0].function_response)
    elif event.content.parts is not None and event.content.parts[0].function_call:
        print(event.content.parts[0].function_call)
    else:
        print(event.content.parts)

0
model
[Part(
  text="""Recent advancements in quantum computing promise to revolutionize AI by offering exponential improvements in processing power and efficiency. These advancements include breakthroughs in quantum hardware, such as the development of the world's first 10,000-qubit quantum processor, and progress in quantum error correction, with the demonstration of a logical qubit prototype.

The implications for AI are significant:

*   **Enhanced AI Capabilities**: Quantum computing can accelerate AI tasks like data mining, pattern recognition, and complex simulations, which are currently limited by classical computing power. This could lead to faster and more accurate AI models, capable of handling exponentially more possibilities for optimization problems.
*   **Solving Complex Problems**: Quantum AI is expected to tackle problems that are beyond the reach of current classical computers, such as drug discovery, materials science, and molecular modeling. This could revolutioni

--- 
## Section 6. Summary - Choosing the Right Pattern

### Decision Tree: Which Workflow Pattern?

<!--
```mermaid
graph TD
    A{"What kind of workflow do you need?"} -- > B["Fixed Pipeline<br>(A → B → C)"];
    A -- > C["Concurrent Tasks<br>(Run A, B, C all at once)"];
    A -- > D["Iterative Refinement<br>(A ⇆ B)"];
    A -- > E["Dynamic Decisions<br>(Let the LLM decide what to do)"];

    B -- > B_S["Use <b>SequentialAgent</b>"];
    C -- > C_S["Use <b>ParallelAgent</b>"];
    D -- > D_S["Use <b>LoopAgent</b>"];
    E -- > E_S["Use <b>LLM Orchestrator</b><br>(Agent with other agents as tools)"];

    style B_S fill:#f9f,stroke:#333,stroke-width:2px
    style C_S fill:#ccf,stroke:#333,stroke-width:2px
    style D_S fill:#cff,stroke:#333,stroke-width:2px
    style E_S fill:#cfc,stroke:#333,stroke-width:2px
```
-->

<img width="1000" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/agent-decision-tree.png" alt="Agent Decision Tree" />

### Quick Reference Table

| Pattern | When to Use | Example | Key Feature |
|---------|-------------|---------|-------------|
| **LLM-based (AgentTool)** | Dynamic orchestration needed | Research + Summarize | LLM decides what to call |
| **Sequential** | Order matters, linear pipeline | Outline → Write → Edit | Deterministic order |
| **Parallel** | Independent tasks, speed matters | Multi-topic research | Concurrent execution |
| **Loop** | Iterative improvement needed | Writer + Critic refinement | Repeated cycles |